## Data Preparation

In [1]:
import pandas as pd
import ast

csv_file_path = 'https://raw.githubusercontent.com/emitterl/AA-project-team-6/main/charging_sessions.csv'

df = pd.read_csv(csv_file_path, delimiter=',', quotechar='"')

### Help functions

In [2]:
# Function for creating the table userInput 
def parse_user_inputs(row):
    
    user_inputs = row['userInputs']
    id = row['id']

    if isinstance(user_inputs, str):
        try:
            # Konvertieren des Strings in ein Python-Dictionary
            user_inputs_data = ast.literal_eval(user_inputs)

            for entry in user_inputs_data:
                entry['reference_id'] = id
            
            return user_inputs_data
        except Exception as e:
            print(f"Fehler beim Parsen von userInputs für ID {id}: {e}")
            return []
    else:
        return []
    

# Function for converting the time zone
def convert_timezone(time):
    if pd.notna(time):
        return time.tz_convert('America/Los_Angeles')
    return time

### Converting the data types in charging_sessions

In [3]:
df['id'] = df['id'].astype(str)
df['connectionTime'] = pd.to_datetime(df['connectionTime'], utc=True, errors='coerce').apply(convert_timezone)
df['disconnectTime'] = pd.to_datetime(df['disconnectTime'], utc=True, errors='coerce').apply(convert_timezone)
df['doneChargingTime'] = pd.to_datetime(df['doneChargingTime'], utc=True, errors='coerce').apply(convert_timezone)
df['kWhDelivered'] = df['kWhDelivered'].astype(float)
df['sessionID'] = df['sessionID'].astype(str)
df['siteID'] = df['siteID'].astype(str)
df['spaceID'] = df['spaceID'].astype(str)
df['stationID'] = df['stationID'].astype(str)
df['timezone'] = df['timezone'].astype(str)
df['userID'] = df['userID'].astype(str)

### Cleaning the data set - main DataFrame

In [4]:
# Delete columns Unnamed, stationID, sessionID   
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.drop(['sessionID', 'stationID', 'timezone'], axis=1, inplace=True)

# Delete all duplicates
df = df.drop_duplicates()

# Set doneChargingTime to disconnectTime wherever doneChargingTime is later than disconnectTime
df.loc[df['doneChargingTime'] > df['disconnectTime'], 'doneChargingTime'] = df['disconnectTime']

# Delete doneChargingTime wherever it is before connectionTime
df.loc[df['doneChargingTime'] < df['connectionTime'], 'doneChargingTime'] = pd.NaT

# Create DataFrame user_inputs_df
user_inputs_list = df.apply(parse_user_inputs, axis=1)
user_inputs_df = pd.DataFrame([item for sublist in user_inputs_list for item in sublist])

# Delete userInputs, as they are in their own table
df = df.drop('userInputs', axis=1)

# Delete paymentRequired, because it is always true
user_inputs_df = user_inputs_df.drop('paymentRequired', axis=1)

# Handle missing values
dfNan = df[df.isna().any(axis=1)]

### Converting the data types in user_inputs_df

In [5]:
user_inputs_df['WhPerMile'] = user_inputs_df['WhPerMile'].astype(float)
user_inputs_df['kWhRequested'] = user_inputs_df['kWhRequested'].astype(float)
user_inputs_df['milesRequested'] = user_inputs_df['milesRequested'].astype(float)
user_inputs_df['minutesAvailable'] = user_inputs_df['minutesAvailable'].astype(float)
user_inputs_df['modifiedAt'] = pd.to_datetime(user_inputs_df['modifiedAt'], utc=True).apply(convert_timezone)
user_inputs_df['requestedDeparture'] = pd.to_datetime(user_inputs_df['requestedDeparture'], utc=True).apply(convert_timezone)

### Cleaning the data set - user inputs DataFrame

In [6]:
user_inputs_df = user_inputs_df.drop('userID', axis=1, errors='ignore')

# Sort the user_inputs_df by 'reference_id' and 'modifiedAt' to get the latest entry for each ID
user_inputs_df_sorted = user_inputs_df.sort_values(by=['reference_id', 'modifiedAt'], ascending=[True, False])

# Keep only the most recent entry for each reference_id
user_inputs_df_latest = user_inputs_df_sorted.drop_duplicates(subset=['reference_id'])

# Merge the df with user_inputs_df_latest
# Left join to ensure that all lines from df are retained
merged_df = pd.merge(df, user_inputs_df_latest, how='left', left_on='id', right_on='reference_id')

# Remove the 'reference_id' column as it is identical to 'id'
merged_df.drop('reference_id', axis=1, inplace=True)

# Removal of outliers
merged_df.loc[merged_df.WhPerMile > 474.8, "WhPerMile"] = 474.8 # https://ev-database.org --> max 474.8 wh/mile
merged_df.loc[merged_df.WhPerMile < 223.7, "WhPerMile"] = 223.7 # https://ev-database.org --> min 223.7 wh/mile
merged_df.loc[merged_df.kWhRequested > 123, "kWhRequested"] = 123 # https://ev-database.org --> max 123 kwh
merged_df = merged_df.drop(merged_df[merged_df.kWhRequested == 0].index, axis=0) #--> 0 not possible --> Del
merged_df = merged_df.drop(merged_df[merged_df.milesRequested == 0].index, axis=0) #--> 0 not possible --> Del
merged_df.loc[merged_df.milesRequested > 425.6, "milesRequested"] = 425.6  #--> # https://ev-database.org --> max 425,6 miles
merged_df = merged_df.drop(merged_df[merged_df.minutesAvailable == merged_df.minutesAvailable.max()].index, axis=0) # Outlier --> Del


%store merged_df

Stored 'merged_df' (DataFrame)
